In [33]:
from sklearn.metrics import mean_squared_error
# from sklearn.ensemble import HistGradientBoostingRegressor as hgbr
# from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
# import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
# import matplotlib.pyplot as plt
# import seaborn as sns
# from supervised.automl import AutoML
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
    ExperimentConfig,
)

In [3]:
main_data_path = "../../data/main_data/"
nulls_path = "../../data/nulls.csv"
other_data_path = "../../data/other_data/"
label_path = "../../data/training_label.csv"
feature_imp_path = "../../data/feature_importance.parquet"
submission_path = "../../data/sample submission.csv"
pred_cols = ["Mean_BMI","Median_BMI","Unmet_Need_Rate","Under5_Mortality_Rate","Skilled_Birth_Attendant_Rate","Stunted_Rate"]

In [4]:
feat = pd.read_parquet(feature_imp_path)
# extract first 500 features
imp_features = feat['feature'][:500].values.tolist()
imp_features

/var/folders/qb/stptmtq13_9gt1v1pzd4g4s00000gn/T/ipykernel_8486/2949786.py:3: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  imp_features = feat['feature'][:500].values.tolist()


['onehotencoder__URBAN_RURA_R',
 'onehotencoder__URBAN_RURA_U',
 'remainder__DHSYEAR',
 'Retrieved_Temperature_Profile_Mean_Mean_780_min_max@MODIS/061/MOD08_M3&timestamped',
 'remainder__LONGNUM',
 'ozone_tropospheric_vertical_column_median@COPERNICUS/S5P/OFFL/L3_O3_TCL&timestamped',
 'gaugeRelativeWeighting_median@NASA/GPM_L3/IMERG_MONTHLY_V06&timestamped',
 'urban-coverfraction_mean@COPERNICUS/Landcover/100m/Proba-V-C3/Global&timestamped',
 'Retrieved_Temperature_Profile_Mean_Mean_950_mean@MODIS/061/MOD08_M3&timestamped',
 'Retrieved_Temperature_Profile_Mean_Mean_780_min_min@MODIS/061/MOD08_M3&timestamped',
 'Retrieved_Temperature_Profile_Mean_Mean_920_mean@MODIS/061/MOD08_M3&timestamped',
 'gaugeRelativeWeighting_mean@NASA/GPM_L3/IMERG_MONTHLY_V06&timestamped',
 'Retrieved_Temperature_Profile_Mean_Mean_920_min_max@MODIS/061/MOD08_M3&timestamped',
 'Retrieved_Temperature_Profile_Mean_Mean_920_min_min@MODIS/061/MOD08_M3&timestamped',
 'onehotencoder__DHSCC_TZ',
 'onehotencoder__DHSCC_

In [34]:
def mcrmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return np.average(rmse)

In [5]:
df = pd.read_parquet(main_data_path)

In [6]:
nulls = pd.read_csv(nulls_path)
nulls = nulls[nulls['0'] > 0.0]['Unnamed: 0'].values
df.drop(columns=nulls, inplace=True)
df = df[~df.index.duplicated(keep='first')]

In [7]:
df.drop(columns=[col for col in df.columns if col not in imp_features], inplace=True)
df

,Ei_median@CAS/IGSNRR/PML/V2&timestamped,Es_median@CAS/IGSNRR/PML/V2&timestamped,Es_mean@CAS/IGSNRR/PML/V2&timestamped,qc_mean@CAS/IGSNRR/PML/V2&timestamped,Ec_min_max@CAS/IGSNRR/PML/V2&timestamped,Ei_max_max@CAS/IGSNRR/PML/V2&timestamped,Es_max_max@CAS/IGSNRR/PML/V2&timestamped,Es_max_min@CAS/IGSNRR/PML/V2&timestamped,Es_min_max@CAS/IGSNRR/PML/V2&timestamped,Es_min_min@CAS/IGSNRR/PML/V2&timestamped,...,motorized_only_friction_surface_20000,motorized_only_travel_time_20000,walking_only_friction_surface_50000,walking_only_travel_time_50000,motorized_only_friction_surface_50000,motorized_only_travel_time_50000,walking_only_friction_surface_100000,walking_only_travel_time_100000,motorized_only_friction_surface_100000,motorized_only_travel_time_100000
DHSID,,,,,,,,,,,,,,,,,,,,,
AL200800000001,0.051240,0.720611,0.703046,0.000000,0.127001,0.442405,2.063823,1.245106,0.079005,0.043457,...,0.011207,16.502903,0.031108,226.565900,0.025174,52.417587,0.043573,216.058010,0.014725,47.410877
AL200800000002,0.072052,0.641110,0.636434,0.000000,0.091355,0.350706,1.924065,0.924236,0.090663,0.051266,...,0.010277,34.382580,0.035934,225.833650,0.029437,56.164467,0.039097,200.149830,0.012322,38.932705
AL200800000003,0.040593,0.809169,0.764483,0.000000,0.110454,0.325511,1.932968,0.903424,0.103369,0.065449,...,0.010316,27.699300,0.032862,225.121170,0.026694,52.411550,0.039717,214.108180,0.013806,43.192800
AL200800000004,0.056218,0.680800,0.676971,0.000000,0.093540,0.367744,2.046383,1.363256,0.085710,0.042362,...,0.010585,16.992321,0.031864,233.255630,0.025747,53.518450,0.043162,216.461870,0.014495,46.634487
AL200800000005,0.072462,0.696633,0.634081,0.000000,0.133332,0.346370,2.406913,1.116209,0.096352,0.045622,...,0.009745,20.305431,0.034980,249.495010,0.027163,58.282570,0.043852,206.457570,0.012748,43.546497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NG200800000718,0.106839,0.401908,1.005747,15.376702,0.168697,0.414405,4.735641,1.670800,0.000000,0.000000,...,0.006625,4.831565,0.025412,52.205936,0.014123,23.250920,0.053384,122.062160,0.007324,62.738655
NG200800000719,0.000000,0.007230,1.260059,16.639208,0.000000,0.000000,4.847148,4.638987,0.000000,0.000000,...,0.004218,3.115215,0.022869,49.061897,0.013825,21.189968,0.050933,109.916050,0.008096,58.283382
NG200800000720,0.162277,0.655050,0.617611,13.818388,-1.000000,-1.000000,-0.987945,-1.000000,-1.000000,-1.000000,...,0.002345,1.880285,0.021904,52.027435,0.012604,24.944271,0.049486,108.954930,0.008825,58.816376


In [9]:
other_df = pd.read_parquet(other_data_path)
other_df = other_df[~other_df.index.duplicated(keep='first')]

In [ ]:
# transformer = make_column_transformer(
#     (OneHotEncoder(), ['DHSCC', 'URBAN_RURA', 'DHSREGNA']),
#     remainder='passthrough')

# transformed = transformer.fit_transform(other_df)
# transformed_df = pd.DataFrame(transformed.todense(), columns=transformer.get_feature_names_out())
# transformed_df.index = other_df.index

In [ ]:
# transformed_df.drop(columns=[col for col in transformed_df.columns if col not in imp_features], inplace=True)
# transformed_df

In [10]:
df = df.join(other_df, how='left')

In [13]:
labels = pd.read_csv(label_path, index_col='DHSID')
labels.drop(columns=['URBAN_RURA', 'DHSYEAR', 'DHSCLUST', 'LATNUM', 'LONGNUM'], inplace=True)

In [14]:
df = df.join(labels, how='left')

In [15]:
y = df['Unmet_Need_Rate']
X = df.drop(columns=pred_cols)

# get rows in y that have all null values
null_rows = y[y.isna()].index
X.drop(index=null_rows, inplace=True)
y.drop(index=null_rows, inplace=True)

In [29]:
cols = [col for col in X.columns if col not in ['DHSCC', 'URBAN_RURA', 'DHSREGNA']]

In [16]:
from sklearn.model_selection import train_test_split

test_size = 0.2
val_size = 0.2
random_state = 42

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size, random_state=random_state)

In [26]:
val = pd.concat([X_val, y_val], axis=1)
train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [ ]:
# y.plot(kind='box')

In [ ]:
# #FUNCTION TO IDENTIFY OUTLIERS USING IQR METHOD
# def iqr_outlier(x,factor):
#     q1 = x.quantile(0.25)
#     q3 = x.quantile(0.75)
#     iqr = q3 - q1
#     min_ = q1 - factor * iqr
#     max_ = q3 + factor * iqr
#     result_ = pd.Series([0] * len(x))
#     result_[((x < min_) | (x > max_)).values] = 1
#     return result_

# iqr_outlier(y, 1.5).value_counts()

In [ ]:
# y_temp = y.reset_index()
# y_temp.plot(kind='scatter', x='DHSID', y='Unmet_Need_Rate')

In [ ]:
# y.plot(kind='hist')

In [ ]:
# # KFold training of Random Forest model

# kf = KFold(n_splits=2, shuffle=True, random_state=42)
# scores = []
# for train_index, test_index in kf.split(X):
#     X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]
#     model = hgbr()
#     # rf = RandomForestRegressor(n_estimators=7000, max_depth=20, max_features=0.5, random_state=42, n_jobs=-1, verbose=1)
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_test)
#     scores.append(mcrmse(y_test, y_pred))
#     print(mcrmse(y_test, y_pred))

# score = np.average(scores)
# score

In [39]:
data_config = DataConfig(
    target=[
        "Unmet_Need_Rate"
    ],  # target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=cols,
    categorical_cols=["DHSCC", "DHSREGNA", "URBAN_RURA"],
)
trainer_config = TrainerConfig(
    auto_lr_find=True,  # Runs the LRFinder to automatically derive a learning rate
    batch_size=1024,
    max_epochs=100,
)
optimizer_config = OptimizerConfig()

model_config = CategoryEmbeddingModelConfig(
    task="regression",
    layers="1024-256-256",  # Number of nodes in each layer
    activation="LeakyReLU",  # Activation between each layers
    learning_rate=1e-3,
)

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)
tabular_model.fit(train=train, validation=val)
result = tabular_model.evaluate(test)
pred_df = tabular_model.predict(test)
# tabular_model.save_model("./examples/basic")
# loaded_model = TabularModel.load_from_checkpoint("./examples/basic")

2023-07-24 01:25:46,616 - {pytorch_tabular.tabular_model:105} - INFO - Experiment Tracking is turned off
Global seed set to 42
2023-07-24 01:25:46,686 - {pytorch_tabular.tabular_model:473} - INFO - Preparing the DataLoaders
2023-07-24 01:25:46,894 - {pytorch_tabular.tabular_datamodule:290} - INFO - Setting up the datamodule for regression task
2023-07-24 01:25:51,922 - {pytorch_tabular.tabular_model:521} - INFO - Preparing the Model: CategoryEmbeddingModel
/opt/miniconda3/envs/ml2/lib/python3.10/site-packages/pytorch_tabular/models/base_model.py:140: UserWarning: Wandb is not installed. Please install wandb to log logits. You can install wandb using pip install wandb or install PyTorch Tabular using pip install pytorch-tabular[all]
  warnings.warn(
/opt/miniconda3/envs/ml2/lib/python3.10/site-packages/pytorch_tabular/models/base_model.py:147: UserWarning: Plotly is not installed. Please install plotly to log logits. You can install plotly using pip install plotly or install PyTorch Tab

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

/opt/miniconda3/envs/ml2/lib/python3.10/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
/opt/miniconda3/envs/ml2/lib/python3.10/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
LR finder stopped early after 90 steps due to diverging loss.
Learning rate set to 0.0005248074602497723
Restoring states from the checkpoint path at /Users/rushilv/Documents/Programs/Stanford_FT/src/column_wise_models/.lr_find_74d79fc6-c577-4e0f-bb5a-3dfa32ad58b3.ckpt
Restored all states from the checkpoint file at /Users/rushilv/Documents/Programs/Stanford_FT/src/column_wise_models/.lr_find_74d79fc6-c577-4e0f-bb5a-3dfa32ad58b3.ckpt
2023-07-24 01:

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  902 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │ 82.9 K │
│ 2 │ head             │ LinearHead                │    257 │
│ 3 │ loss             │ MSELoss                   │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 985 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 985 K                                                                                                
Total estimated model params size (MB): 3

...

/opt/miniconda3/envs/ml2/lib/python3.10/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You called
`self.log('test_loss', ..., logger=True)` but have no logger configured. You can enable one by doing 
`Trainer(logger=ALogger(...))`
  rank_zero_warn(

/opt/miniconda3/envs/ml2/lib/python3.10/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You called
`self.log('test_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by doing 
`Trainer(logger=ALogger(...))`
  rank_zero_warn(

/opt/miniconda3/envs/ml2/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │     405.9709777832031     │
│  test_mean_squared_error  │     405.9709777832031     │
└───────────────────────────┴───────────────────────────┘

Output()

In [40]:
pred_df

,Ei_median@CAS/IGSNRR/PML/V2&timestamped,Es_median@CAS/IGSNRR/PML/V2&timestamped,Es_mean@CAS/IGSNRR/PML/V2&timestamped,qc_mean@CAS/IGSNRR/PML/V2&timestamped,Ec_min_max@CAS/IGSNRR/PML/V2&timestamped,Ei_max_max@CAS/IGSNRR/PML/V2&timestamped,Es_max_max@CAS/IGSNRR/PML/V2&timestamped,Es_max_min@CAS/IGSNRR/PML/V2&timestamped,Es_min_max@CAS/IGSNRR/PML/V2&timestamped,Es_min_min@CAS/IGSNRR/PML/V2&timestamped,...,motorized_only_friction_surface_100000,motorized_only_travel_time_100000,DHSCC,DHSREGNA,DHSYEAR,LATNUM,LONGNUM,URBAN_RURA,Unmet_Need_Rate,Unmet_Need_Rate_prediction
DHSID,,,,,,,,,,,,,,,,,,,,,
HT201200000120,0.131799,1.245004,1.169833,0.034934,1.217672,0.576755,6.003401,0.849503,0.043200,0.031370,...,0.003160,-1.000000,HT,Aire Metropolitaine/ Reste-Ouest,2012,18.532747,-72.441910,U,50.00,45.137611
PH201700000091,-1.000000,-1.000000,-0.997732,34.000000,-1.000000,-1.000000,-0.987945,-1.000000,-1.000000,-1.000000,...,0.003160,-1.000000,PH,Cordillera Admin Region,2017,0.000000,0.000000,R,15.38,42.774384
HT200600000044,0.000000,2.540594,1.873050,2.104753,2.118051,0.348493,5.920708,1.444297,0.034034,0.024437,...,0.003160,-1.000000,HT,sud,2006,18.195295,-73.742455,U,41.18,45.913395
TG201300000118,0.201830,0.685209,0.700195,2.608868,0.608650,0.353353,1.466618,1.315017,0.000000,0.000000,...,0.011044,23.135725,TG,centrale,2013,8.211965,0.955900,R,45.45,53.532608
NI199200000229,0.002225,0.239549,0.864013,0.833879,0.399316,0.157701,3.712190,2.567163,0.000260,0.000193,...,0.011660,12.567299,NI,zinder,1992,13.033300,9.616700,R,100.00,81.463936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KH201400000209,0.105786,1.745129,1.660871,4.095869,1.436099,0.788917,3.710885,1.884544,0.157450,0.129978,...,0.007543,34.431774,KH,kratie,2014,12.485047,105.992780,R,9.09,28.756372
BD201100000343,0.082950,0.924371,0.961432,3.514931,0.683223,0.854225,2.949728,2.112955,0.000000,0.000000,...,0.005178,10.466812,BD,Khulna,2011,23.866575,88.829315,R,0.00,19.180504
DR200700002092,0.427800,0.654783,0.654821,0.008238,1.204705,1.122669,1.452781,0.614580,0.098831,0.075536,...,0.008845,44.757645,DR,viii,2007,19.009706,-70.631780,R,14.29,15.320802


In [41]:
mcrmse(pred_df['Unmet_Need_Rate'], pred_df['Unmet_Need_Rate_prediction'])

20.148720682198512

In [42]:
sub = pd.read_csv(submission_path, index_col='DHSID')
df.drop(columns=pred_cols, inplace=True)
test = sub.join(df, how='left')
test.drop(columns=pred_cols, inplace=True)
test

,Ei_median@CAS/IGSNRR/PML/V2&timestamped,Es_median@CAS/IGSNRR/PML/V2&timestamped,Es_mean@CAS/IGSNRR/PML/V2&timestamped,qc_mean@CAS/IGSNRR/PML/V2&timestamped,Ec_min_max@CAS/IGSNRR/PML/V2&timestamped,Ei_max_max@CAS/IGSNRR/PML/V2&timestamped,Es_max_max@CAS/IGSNRR/PML/V2&timestamped,Es_max_min@CAS/IGSNRR/PML/V2&timestamped,Es_min_max@CAS/IGSNRR/PML/V2&timestamped,Es_min_min@CAS/IGSNRR/PML/V2&timestamped,...,walking_only_friction_surface_100000,walking_only_travel_time_100000,motorized_only_friction_surface_100000,motorized_only_travel_time_100000,DHSCC,DHSREGNA,DHSYEAR,LATNUM,LONGNUM,URBAN_RURA
DHSID,,,,,,,,,,,,,,,,,,,,,
AL200800000003,0.040593,0.809169,0.764483,0.000000,0.110454,0.325511,1.932968,0.903424,0.103369,0.065449,...,0.039717,214.10818,0.013806,43.192800,AL,central,2008.0,40.750040,19.974262,R
AL200800000005,0.072462,0.696633,0.634081,0.000000,0.133332,0.346370,2.406913,1.116209,0.096352,0.045622,...,0.043852,206.45757,0.012748,43.546497,AL,central,2008.0,40.746124,19.843885,R
AL200800000007,0.057890,0.727580,0.686621,0.000000,0.119398,0.300231,2.107771,1.250919,0.098821,0.055466,...,0.039564,203.26802,0.012489,39.679590,AL,central,2008.0,40.701607,19.989952,U
AL200800000008,0.025905,0.843817,0.968380,0.000000,0.140053,0.576523,3.575065,1.014970,0.105989,0.063329,...,0.040597,202.23082,0.012181,39.403683,AL,central,2008.0,40.695984,19.965063,U
AL200800000009,0.039826,0.784812,0.785890,0.000000,0.109034,0.287542,3.549297,1.252482,0.101052,0.055466,...,0.040002,201.76999,0.012244,39.236156,AL,central,2008.0,40.698685,19.981623,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZW201500000382,0.014547,0.592953,0.828747,0.388586,0.453606,0.282589,2.533496,1.745731,0.012040,0.008606,...,0.014643,140.47174,0.012210,67.942795,ZW,Masvingo,2015.0,-19.939451,31.822948,R
ZW201500000383,0.004192,0.570845,0.591410,0.228348,0.331372,0.414315,1.918416,1.627009,0.000000,0.000000,...,0.014095,143.71710,0.011881,50.976006,ZW,Matabeleland North,2015.0,-19.851952,28.240479,R
ZW201500000386,0.000800,0.543463,0.648765,0.228583,1.101482,0.654535,4.975616,1.220297,0.000000,0.000000,...,0.016248,212.91324,0.009727,30.100767,ZW,Matabeleland South,2015.0,-22.126260,29.639608,R


In [ ]:
# impute test values
imputer = KNNImputer(n_neighbors=75)
test = pd.DataFrame(imputer.fit_transform(test), columns=test.columns, index=test.index)
test

In [ ]:
preds = automl.predict(test)
preds

In [ ]:
final = pd.DataFrame(preds, columns=['Unmet_Need_Rate'], index=sub.index)
final

In [ ]:
final.to_parquet('./unmet_need_rate.parquet.gzip', compression='gzip')